In [1]:
import pandas as pd
import requests
import altair as alt

collections = ["undervalued_growth_stocks",
               "growth_technology_stocks",
               "undervalued_large_caps",
               "aggressive_small_caps",
               "small_cap_gainers"]

# Initialize lists to store DataFrames for PE Ratio and Book Value
pe_dfs = []
bv_dfs = []

# Initialize a list to store box plots
box_plots = []

for collection in collections:
    startIndex = None
    if (collection == "undervalued_large_caps"):
        startIndex = 1
    else:
        startIndex = 0
    # replace the "demo" apikey below with your own key
    url = f"https://mboum.com/api/v1/co/collections/?list={collection}&start={startIndex}&apikey=demo"
    r = requests.get(url)
    data = r.json()
    quotes = data['data']['quotes']

    # Initialize lists to store data
    symbols = []
    pe_ratios = []
    book_values = []

    for quote in quotes:
        symbols.append(quote['symbol'])
        pe_ratios.append(quote.get('forwardPE', None))
        book_values.append(quote.get('priceToBook', None))

    # Create DataFrame for the current collection
    pe_df = pd.DataFrame({
        'Collection': [collection.replace("_", " ").title()] * len(symbols),
        'Symbol': symbols,
        'PE Ratio': pe_ratios
    })

    bv_df = pd.DataFrame({
        'Collection': [collection.replace("_", " ").title()] * len(symbols),
        'Symbol': symbols,
        'Book Value': book_values
    })

    pe_dfs.append(pe_df)
    bv_dfs.append(bv_df)

    # Combine all DataFrames into single DataFrames for PE Ratio and Book Value
    combined_pe_df = pd.concat(pe_dfs)
    combined_bv_df = pd.concat(bv_dfs)

    # Define the color scheme with different shades of green
    color_scheme = alt.Scale(range=["#238b45", "#41ab5d", "#78c679", "#addd8e", "#d9f0a3"])


    # Create a single box plot for all PE Ratios
    pe_box_plot = alt.Chart(combined_pe_df).mark_boxplot().encode(
        x=alt.X('Collection:N', title=None),  # Group by collection
        y=alt.Y('PE Ratio:Q', axis=alt.Axis(title='PE Ratio')),  # Y-axis for PE Ratio
        color=alt.Color('Collection:N', legend=None, scale=color_scheme)  # Color by collection
    ).properties(
        width=500,  # Set the width of the chart
        height=400,  # Set the height of the chart
        title='PE Ratio Across Collections'  # Set the title of the chart
        )
    
    # Create a single box plot for all Book Values
    bv_box_plot = alt.Chart(combined_bv_df).mark_boxplot().encode(
        x=alt.X('Collection:N', title=None),  # Group by collection
        y=alt.Y('Book Value:Q', axis=alt.Axis(title='Book Value'), scale=alt.Scale(domain=[-20, 70])),  # Y-axis for Book Value
        color=alt.Color('Collection:N', legend=None, scale=color_scheme)  # Color by collection
    ).properties(
        width=500,  # Set the width of the chart
        height=400,  # Set the height of the chart
        title='Book Value Across Collections'  # Set the title of the chart
        )


# Combine box plots into a single chart
combined_plots = pe_box_plot | bv_box_plot

# Customize chart appearance
combined_plots = combined_plots.configure_title(
    fontSize=16,  # Set title font size
    fontWeight='bold',  # Set title font weight
    color='#006400'  # Set title color
).configure_axis(
    labelFontSize=12,  # Set axis label font size
    titleFontSize=14,  # Set axis title font size
    titleFontWeight='normal',  # Set axis title font weight
)

combined_plots




alt.HConcatChart(...)